In [1]:
import go
arg_list = go.setup_running_env()

/home/haoranhuang/anaconda3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
predict_type = "softmax"
runner = go.get_runner(predict_type)
args, config, tokenizer, model, model_classes = runner.setup(arg_list=arg_list)
config_class, model_class, tokenizer_class = model_classes
train_dataset = runner.load_and_cache_examples(args, args.task_name, tokenizer, data_type='train')

08/09/2022 07:42:02 - INFO - run_ner_softmax:463 -  device_name is cuda
08/09/2022 07:42:02 - WARNING - run_ner_softmax:473 -  Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
08/09/2022 07:42:02 - INFO - run_ner_softmax:485 -  task_name is cluener
08/09/2022 07:42:02 - INFO - run_ner_softmax:495 -  model_type is bert
08/09/2022 07:42:02 - INFO - configuration_utils:120 -  
=============Loading configuration from pretrain. kwargs: {'num_labels': 34, 'loss_type': 'ce', 'cache_dir': '/home/haoranhuang/.cache/torch/transformers'}
08/09/2022 07:42:02 - INFO - configuration_utils:151 -  loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /home/haoranhuang/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.f12a4f986e43d8b328f5b067a641064d67b91597567a06c7b122d1ca7dfd9741
08/09/2022 07:42:02 - INFO - configuration_utils:171 -  Model config

In [3]:
len(train_dataset)

for feat in train_dataset:
    if len(feat[0]) != 128:
        print("feat", len(feat[0]))
        break
    if len(feat[1]) != 128:
        print("feat", len(feat[0]))
        break
    if len(feat[2]) != 128:
        print("feat", len(feat[0]))
        break
#     empty tensor
#     print(feat[3][0])
    if len(feat[4]) != 128:
        print("feat", len(feat[0]))
        break


In [4]:
print(args)

Namespace(task_name='cluener', data_dir='/home/haoranhuang/workspace/CLUENER2020/pytorch_version/CLUEdatasets/cluener', model_type='bert', model_name_or_path='bert-base-chinese', output_dir='/home/haoranhuang/workspace/CLUENER2020/pytorch_version/outputs/bert-base-chinese/cluener_output/bert', markup='bios', loss_type='ce', labels='', config_name='', tokenizer_name='', cache_dir='/home/haoranhuang/.cache/torch/transformers', train_max_seq_length=128, eval_max_seq_length=512, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=False, do_lower_case=True, per_gpu_train_batch_size=24, per_gpu_eval_batch_size=24, gradient_accumulation_steps=1, learning_rate=3e-05, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=4.0, max_steps=-1, warmup_steps=0, logging_steps=224, save_steps=224, eval_all_checkpoints=False, predict_checkpoints=0, no_cuda=False, overwrite_output_dir=True, overwrite_cache=False, seed=42, fp16=False, fp16_opt_level='O1', local_rank=

In [8]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from processors.ner_seq import batch_collate_fn


# args, train_dataset, model, tokenizer
train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.per_gpu_train_batch_size,
                              collate_fn=batch_collate_fn)

In [9]:
torch.device("cuda")

device(type='cuda')

In [32]:
import torch

import tools.common as common

cpu = common.get_device("cpu")
mps = common.get_device("mps")
cuda = common.get_device("cuda")
target_device = cuda

import time
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=1,
                              collate_fn=batch_collate_fn)


# convert batch to input list
inputs_list = []
for step, batch in enumerate(train_dataloader):
#     print("======")
#     print(f"batch is\n {batch}")
#     print(f"len(batch) is\n {len(batch)}")
#     print(f"batch.shape is\n {batch[0].shape}")
#     print(batch[0].shape)
#     print(batch[1].shape)
#     print(batch[2].shape)
#     print(batch[3].shape)
#     print(batch[4].shape)
    
    inputs = {
        "input_ids": batch[0].to(target_device),
        "attention_mask": batch[1].to(target_device),
        "labels": batch[3].to(target_device),
    }
    if args.model_type != "distilbert":
        # XLM and RoBERTa don"t use segment_ids
        inputs["token_type_ids"] = (batch[2].to(target_device) if args.model_type in ["bert", "xlnet"] else None)
    inputs_list.append(inputs)
    break


In [33]:
from processors.ner_seq import ner_processors as processors
processor = processors["cluener"]()
label_map = processor.get_labels()
id2label = {i:label for i, label in enumerate(label_map)}
print(id2label)

{0: 'X', 1: 'B-address', 2: 'B-book', 3: 'B-company', 4: 'B-game', 5: 'B-government', 6: 'B-movie', 7: 'B-name', 8: 'B-organization', 9: 'B-position', 10: 'B-scene', 11: 'I-address', 12: 'I-book', 13: 'I-company', 14: 'I-game', 15: 'I-government', 16: 'I-movie', 17: 'I-name', 18: 'I-organization', 19: 'I-position', 20: 'I-scene', 21: 'S-address', 22: 'S-book', 23: 'S-company', 24: 'S-game', 25: 'S-government', 26: 'S-movie', 27: 'S-name', 28: 'S-organization', 29: 'S-position', 30: 'S-scene', 31: 'O', 32: '[CLS]', 33: '[SEP]'}


In [34]:
model.zero_grad()
model.train()
def myforward(self, input_ids, attention_mask=None, token_type_ids=None,
            position_ids=None, head_mask=None, labels=None):
    outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    sequence_output = outputs[0]
    pooled_sequence_output = outputs[1]
    # sequence_output is the hidden states of all tokens (all = length of sentence)
    # For example: torch.Size([1, 51, 768])
    # pooled_sequence_output the hidden states of the first token
    # pooled_sequence_output is torch.Size([1, 768])
    print(f"sequence_output is {sequence_output.shape}")
    print(f"pooled_sequence_output is {pooled_sequence_output.shape}")
    
    # apply drop out in sequence output
    sequence_output = self.dropout(sequence_output)
    logits = self.classifier(sequence_output)
    print(f"logits is {logits.shape}")
    # print the softmax of first token's prediction
    print(logits[0][0])
    

# print(model.bert)
outputs = myforward(model, **inputs_list[0])
# print()

sequence_output is torch.Size([1, 41, 768])
pooled_sequence_output is torch.Size([1, 768])
logits is torch.Size([1, 41, 34])
tensor([ 0.4948, -0.3017, -0.8894, -0.0891,  0.3375,  0.1482,  0.2054,  0.6662,
        -0.4130,  0.1322, -0.3798, -0.2816,  0.0759,  0.3360,  0.2200, -0.0563,
         0.3037, -0.2275,  0.4163, -0.3351, -0.1387,  0.3301,  0.1531,  0.4575,
         0.3724, -0.0987,  0.0260, -0.5006, -0.2478, -0.3950,  0.4386,  0.3392,
        -0.1641,  0.0565], device='cuda:0', grad_fn=<SelectBackward0>)
